In [1]:
# Nativos
import random as rn
import os
import sys
import gc
import time

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]


In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO_MENOR_0.5,_aux_TRX_SUBIDA_30PERCENT_TRANSFERENCIAS__CASH_OUT_MES_ANT,_75_cut_up_ROTACION_CTASXCOBRAR_VAR_DIAS_CEF_SITUACION,FUNNEL_ALERTA,_aux_TRX_CAIDA_10PERCENT_OPERACIONES_MESA__MES_ANT,DEBT_IN_SF_ULT_RCC_x,_0_cut_up_TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U7M,TIENE_IMPORTE_TRIB_U1M_OVER_U12M_MAYOR_1,ADEX_MAYOR_U3M_IMP,ACTIVIDAD__Comercio,...,_5_cut_up_NUM_MESES_SIN_LINEA_VIGENTE,_2_cut_up_NUM_DOCS_PROT_PROMEDIO_U9M,_1_cut_up_NUM_DOCS_PROT_U6M,_1_cut_up_NUM_DOCS_TRIB_U12M,_2_cut_up_NUM_DOCS_IMPAG_PROMEDIO_U6M,_3_cut_up_NUM_DOCS_IMPAG_U12M,_1_cut_up_NUM_DOCS_LAB_U6M,_1_cut_up_NUM_DOCS_TRIB_PROMEDIO_U9M,cut_letal_SUBSECTOR_s2_provincia,_2_cut_up_NUM_DOCS_IMPAG_PROMEDIO_U9M
0,0.0,0.0,1.0,NaN,0.0,9253118.94,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,NaN,0.0,521627.95,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,NaN,0.0,48350156.96,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,NaN,0.0,24993499.25,0.0,0.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,NaN,0.0,973224.29,0.0,0.0,1.0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
'COD_UNICO' in data.columns, 'PERIODO' in data.columns, 

(True, True)

In [4]:
print(data.shape)
data = filtro_banca_s1provincia(data)
print(data.shape)

(24014, 3638)
(1315, 3638)


In [5]:
pd.crosstab(
    data['PERIODO'], data['ZONAL']
)

ZONAL,BEP ZONAL 1,BEP ZONAL 2,BEP ZONAL 3
PERIODO,,,
202009,99,57,65
202010,100,57,65
202011,97,57,64
202012,98,59,62
202101,97,59,64
202102,95,58,62


### CREACION DE COLUMNAS A MEZCLAR

In [6]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} if '1_and_1' not in col]
print(len(cols_combine))

2352


In [7]:
cols_combine = \
    [col for col in cols_combine if 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and '%' not in col.upper() \
                                 and 'COVID' not in col.upper()
    ]
print(len(cols_combine))

2266


In [8]:
for col in ['cambio_seguir_covid'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

len(cols_combine)

2265

### GENERACION DE ALERTAS

In [9]:
existentes = pd.read_csv('COMBINACIONES_S1_PROVINCIA.csv')
existentes.head()

,,,,,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE
TICKET_PROMEDIO_EN_RIESGO__Bucket_4_-_9.2_MM,NO_TIENE_GAR_AUTO_HIPO_PREN_SF,NaN,6,3,0.5_2_cut_up_DTOTAL_S_EBITDA_CEF_SITUACION,_aux_CASHOUT_CAIDA_40PERCENT_PAGOS_MASIVOS__U6M,NaN,10.0,5.0,0.5
CLASIF_SBS__NORMAL,_5_cut_up_DTOTAL_S_EBITDA_CEF_SITUACION,_0.4_cut_up_minimo_percent_others_u12m,7,3,0.42857142857142855,NaN,NaN,NaN,NaN,NaN
_5_cut_up_DTOTAL_S_EBITDA_CEF_SITUACION,_0.4_cut_up_minimo_percent_others_u12m,NaN,8,4,0.5,NaN,NaN,NaN,NaN,NaN
_1_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,TIENE_AUMENTO_RIESGO_40PERCENT_A_MAS_U3M,NaN,7,3,0.42857142857142855,NaN,NaN,NaN,NaN,NaN
IS_NORMAL_IBK_SBS,_5_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF,NaN,7,3,0.42857142857142855,NaN,NaN,NaN,NaN,NaN


In [10]:
try:
    existentes['union'] = existentes[['VAR_1', 'VAR_2', 'VAR_3']].apply(
        lambda _: '1_and_1_{}___{}'.format(_[0], _[1]) if pd.isnull(_[2]) else '1_and_1_and_1_{}___{}____{}'.format(_[0], _[1], _[2]), 
        axis=1
    )
    listado = list(existentes['union'].unique())
    len(listado)
except:
    listado = []
    
len(listado), data.shape

(30, (1315, 3638))

In [11]:
import random as rn
import itertools

col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'
display(pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
))

aucc = {}
struct = []

data = data[data[col_target].notnull()][cols_combine + ['COD_UNICO', col_target, 'PERIODO']]
data.shape

PERIODO,202009,202010,202011,202012,202101,202102
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,,
-1.0,64,61,57,58,51,49
0.0,149,152,153,154,159,154
1.0,8,9,8,7,10,12


(975, 3638)

In [12]:
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_MEDIANA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S1_PROVINCIA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S2_PROVINCIA.csv', index=False)

In [13]:
SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

rn.shuffle(cols_combine)

In [ ]:
from datetime import datetime
tiempo_inicio = time.time()
nn = len(struct)

while True:
    x = rn.choice(cols_combine)
    y = rn.choice(cols_combine)
    z = rn.choice(cols_combine)
    
    col_evaluate = '1_and_1_and_1_{}___{}____{}'.format(x, y, z)
    
    if col_evaluate not in listado and x != y and x!= z and y!= z:
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1 and _[2] >= 1
        data[col_evaluate] = data[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        #data_play_1[col_evaluate] = data_play_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        
        
        coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
        acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
        
        if coincidencias > 5 and acertados/coincidencias > 0.4:
            aucc[col_evaluate] = acertados/coincidencias
            struct.append(
                [x, y, z, coincidencias, acertados, acertados/coincidencias]
            )
            tiempo_inicio = time.time()
            nn = len(struct)
            #print(struct[-1])
            
        ### Generado pares de 2 
        for a, b in list(itertools.combinations([x, y, z], 2)):
            
            col_evaluate = '1_and_1_{}___{}'.format(a, b)
            if col_evaluate not in listado:
                lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1
                data[col_evaluate] = data[[a, b]].apply(lambda_basico, axis=1).astype(int)
                #data_play_1[col_evaluate] = data_play_1[[a, b]].apply(lambda_basico, axis=1).astype(int)
                
                coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
                acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
                
                if coincidencias > 5 and acertados/coincidencias > 0.4:
                    aucc[col_evaluate] = acertados/coincidencias
                    struct.append(
                        [a, b, None, coincidencias, acertados, acertados/coincidencias]
                    )
                    tiempo_inicio = time.time()
                    nn = len(struct)
                    #print(struct[-1])
        
        if len(struct) >= 5:
            pd.DataFrame(struct).to_csv('COMBINACIONES_S1_PROVINCIA.csv', header=False, index=False, mode='a')
            print("/"*25, datetime.now())
            struct = []
            
        if time.time() - tiempo_inicio >= 3600 and nn == len(struct):
            print("Mas de 1 hora sin nueva combinacion de alerta")
            break
                                                        

///////////////////////// 2021-05-28 04:52:04.775547
///////////////////////// 2021-05-28 04:52:39.668215
///////////////////////// 2021-05-28 04:53:50.685434
///////////////////////// 2021-05-28 04:54:45.901242
///////////////////////// 2021-05-28 04:55:26.048044
///////////////////////// 2021-05-28 04:55:53.402493
///////////////////////// 2021-05-28 04:56:21.964245
///////////////////////// 2021-05-28 04:56:34.802704
///////////////////////// 2021-05-28 04:56:52.788270
///////////////////////// 2021-05-28 04:57:22.929800
///////////////////////// 2021-05-28 04:57:38.049179
///////////////////////// 2021-05-28 04:58:57.907824
///////////////////////// 2021-05-28 04:59:38.534696
///////////////////////// 2021-05-28 05:01:04.938769
///////////////////////// 2021-05-28 05:01:42.874030
///////////////////////// 2021-05-28 05:02:13.007602
///////////////////////// 2021-05-28 05:03:54.957272
///////////////////////// 2021-05-28 05:04:06.209993
///////////////////////// 2021-05-28 05:04:31.